# Session 13: Visualization

In [1]:

import parsl
from parsl import python_app
from parsl.config import Config
from parsl.channels import LocalChannel
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

from ipyleaflet import Map, LocalTileLayer, WMSLayer, projections

from grouputils import initialize_rasterizer

In session 8 and 11, tiles were created at the maximum zoom level (z: 13). To show this data in a performant way on a web map, we need to create lower resolution tiles, and they need to be in a web-accessible format (most browsers can't render GeoTIFF files, PNG is better.) - [Here is a good explanation](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/)

First set up the rasterizer as before.

In [11]:
iwp_rasterizer = initialize_rasterizer("/home/jclark/example-data")

In [7]:
# Set up Parsl and logging again:
activate_env = 'workon scomp'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            worker_debug=False,
            cores_per_worker=1,
            max_workers=26,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=20,
                worker_init=activate_env
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

Here are the methods that we will use from the RasterTiler:

In [8]:
# We'll also use the make_batch definition from Session 11 to create batches of
# GeoTIFF files to process.
def make_batch(items, batch_size):
    """
    Create batches of a given size from a list of items.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

In [9]:
# Here are the parallel tasks we want to run:

@python_app
def create_composite_geotiffs(tiles, rasterizer):
    return rasterizer.parent_geotiffs_from_children(tiles, recursive=False)

@python_app
def create_web_tiles(geotiff_paths, rasterizer):
    return rasterizer.webtiles_from_geotiffs(
        geotiff_paths, update_ranges=False)

To create lower resolution GeoTIFF files, we can combine high resolution GeoTIFFs then resample them so that we still have 256x256 pixel data.

In [12]:
# Get each z-level of GeoTIFFs we need to create:
min_z = iwp_rasterizer.config.get_min_z()
max_z = iwp_rasterizer.config.get_max_z()
parent_zs = list(range(max_z - 1, min_z - 1, -1))
print(f'Parent Zs: {parent_zs}')

Parent Zs: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


## Create composite GeoTIFFs

In [14]:
# Making composite GeoTIFFs is super fast, so let's choose a larger batch size
composite_geotiff_batch_size = 100

# Can't start lower z-level until higher z-level is complete, because tiles in
# z-level n are created from tiles in z-level n+1, e.g. each level 12 tile is
# created from four level 13 tiles.
for z in parent_zs:

    # Determine which tiles we need to make for the next z-level based on
    # the path names of the files just created

    # Get the list of files that exist for the one-step-higher-resolution
    # z-level than the current z-level
    child_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff', z=z + 1)

    # Make a list of all the parent tiles that can be created from the z+1 child
    # tiles
    parent_tiles = set()
    for child_path in child_paths:
        parent_tile =  iwp_rasterizer.tiles.get_parent_tile(child_path)
        parent_tiles.add(parent_tile)
    parent_tiles = list(parent_tiles)

    # Break all parent tiles at level z into batches
    parent_tile_batches = make_batch(parent_tiles, composite_geotiff_batch_size)

    # Make parent GeoTIFFs for the current z-level (by combining the children tiles)
    app_futures = []
    for parent_tile_batch in parent_tile_batches:
        app_future = create_composite_geotiffs(parent_tile_batch, iwp_rasterizer)
        app_futures.append(app_future)

    # Don't start the next z-level, and don't move to step 4, until the
    # current z-level is complete
    [app_future.result() for app_future in app_futures]

# Make webtiles

In [16]:
# Making webtiles is also super fast, so we can big a larger batch size
batch_size_web_tiles = 200

# Get the min & max pixel values that exist across each z-level. Save these to
# the config file.
iwp_rasterizer.update_ranges()

# Process web tiles in batches
geotiff_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff')
geotiff_batches = make_batch(geotiff_paths, batch_size_web_tiles)

app_futures = []
for batch in geotiff_batches:
    app_future = create_web_tiles(batch, iwp_rasterizer)
    app_futures.append(app_future)

# Don't record end time until all web tiles have been created
[app_future.result() for app_future in app_futures]

htex_local.executors[0].shutdown()
parsl.clear()


## Make a map!

In [ ]:
wms = WMSLayer(
    url="https://basemap.nationalmap.gov:443/arcgis/services/USGSImageryTopo/MapServer/WmsServer",
    layers="0",
    format="image/png",
    transparent=True,
    min_zoom=0,
    crs=projections.EPSG4326,  # I'm asking this WMS service to reproject the tile layer using EPSG:4326
)



m = Map(center=(66.5, -159.9),
        zoom=9,
        layers=(wms,),
        crs=projections.EPSG4326)

m.add_layer(LocalTileLayer(path='web_tiles/number_IWP_per_pixel/WorldCRS84Quad/{z}/{x}/{y}.png'));

m